In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/FDM MINI PROJECT/Feature Engineering/

/content/drive/MyDrive/FDM MINI PROJECT/Feature Engineering


In [3]:
# %pip install mlxtend --upgrade
# %pip install sklearn

In [4]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [5]:
%matplotlib inline

In [6]:
df = pd.read_csv(r'cleaned_data_v2.csv')

**Converting categorical values to numerical values using LabelEncoding**

In [7]:
def label_encoding(columns):

  le = LabelEncoder()

  for column in columns:
    label = le.fit_transform(df[column])
    x = np.array(label)

    print(f"column name: {column}")
    print(f"label: {label}")
    print(f"unique values: {np.unique(x)}")
    print()

    df[column] = label

In [9]:
columns = ['meal', 'market_segment', 'distribution_channel', 
           'deposit_type', 'customer_type', 'is_same_room']
label_encoding(columns)

column name: meal
label: [2 0 0 ... 0 0 2]
unique values: [0 1 2 3]

column name: market_segment
label: [5 6 6 ... 6 6 6]
unique values: [0 1 2 3 4 5 6 7]

column name: distribution_channel
label: [3 3 3 ... 3 3 3]
unique values: [0 1 2 3 4]

column name: deposit_type
label: [0 0 0 ... 0 0 0]
unique values: [0 1 2]

column name: customer_type
label: [2 2 2 ... 2 2 2]
unique values: [0 1 2 3]

column name: is_same_room
label: [1 1 1 ... 1 1 1]
unique values: [0 1]



**Filter Method**

In [10]:
corr = df.corr()['is_canceled'].sort_values()
corr

total_of_special_requests        -0.293889
booking_changes                  -0.149500
required_car_parking_spaces      -0.133096
customer_type                    -0.116860
is_repeated_guest                -0.065840
previous_bookings_not_canceled   -0.053134
meal                             -0.041877
babies                           -0.030186
children                         -0.027089
adr                              -0.011965
stays_in_weekend_nights          -0.007343
market_segment                    0.004269
stays_in_week_nights              0.048704
adults                            0.053054
days_in_waiting_list              0.060965
agent                             0.066064
distribution_channel              0.164551
previous_cancellations            0.166643
is_same_room                      0.229972
lead_time                         0.309242
deposit_type                      0.517503
is_canceled                       1.000000
Name: is_canceled, dtype: float64

In [11]:
new_df = df[['is_canceled',
           'lead_time', 
           'is_same_room', 
           'previous_cancellations', 
           'days_in_waiting_list', 
           'adults', 
           'total_of_special_requests', 
           'booking_changes',
           'required_car_parking_spaces',
           'is_repeated_guest',
           'previous_bookings_not_canceled']].copy()

In [12]:
x = new_df.drop('is_canceled', axis=1)
y = new_df['is_canceled']

**Splitting training and testing data set**

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

print('Training dataset shape:', x_train.shape, y_train.shape)
print('Testing dataset shape:', x_test.shape, y_test.shape)

clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

sfs1 = SFS(clf,
           k_features=5,
           forward=False,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)


sfs1 = sfs1.fit(x_train, y_train)

feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)

ModuleNotFoundError: ignored

In [ ]:
final_df = x.iloc[:,feat_cols]
final_df['is_cancelled'] = y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
final_df.head()

lead_time  is_same_room  previous_cancellations  total_of_special_requests  \
0   0.009539             1                       0                          0   
1   0.139905             1                       0                          1   
2   0.103339             1                       0                          1   
3   0.146264             1                       0                          2   
4   0.158983             1                       0                          1   

   booking_changes  is_cancelled  
0                0             0  
1                0             1  
2                0             1  
3                0             1  
4                0             1

In [ ]:
final_df.to_csv('final.csv', index=False)